In [362]:
import os
import sys
import cv2
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

ROOT = os.getcwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from metrics.losses import PatchLoss
from dataset.FAS_dataset import FASDataset
from utils.utils import read_cfg, build_network, get_device

In [363]:
#c:/Users/alitu/Desktop/biometric/Implementation-patchnet/fas_1/Scripts/python.exe -m pip install ipykernel -U --force-reinstall

In [364]:
cfg = read_cfg(cfg_file='config/config.yaml')
device = get_device(cfg)

network = build_network(cfg, device)
network = network.to(device)
loss = PatchLoss().to(device)
saved_name = "/Users/aliturliyeva/Documents/GitHub/patchnet_bio/datasets_cropped/save_160/swin_base_199_1.4580870957246848.pth"
state = torch.load(saved_name)

network.load_state_dict(state['state_dict'])
loss.load_state_dict(state["loss"])
print("loading done!")
network.eval()


/Users/aliturliyeva/Documents/GitHub/patchnet_bio/patchnet/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aliturliyeva/Documents/GitHub/patchnet_bio/patchnet/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


loading done!


FeatureExtractor(
  (nets): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

In [365]:
transform = transforms.Compose([

   # transforms.ToPILImage(),
    transforms.Resize(cfg['model']['image_size']),
    transforms.RandomCrop(cfg['dataset']['augmentation']['rand_crop_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])

])

In [366]:
df_test = pd.read_csv("/Users/aliturliyeva/Documents/GitHub/patchnet_bio/datasets_cropped/test.csv")

In [367]:
df_test['img_path'] = df_test['image_name'].apply(lambda x:'/Users/aliturliyeva/Documents/GitHub/patchnet_bio/datasets_cropped/images/' + x )

In [368]:
result_path = "test_result_patchnet.csv"

In [369]:
def predict(img_path):
    image = Image.open(img_path)
    image = transform(image)
    image = image.unsqueeze(0)
    feature = network.forward(image)
    feature = F.normalize(feature, dim=1)
    score = F.softmax(loss.amsm_loss.s * loss.amsm_loss.fc(feature.squeeze()), dim=0)
    result = torch.argmax(score)
    return [score, result]


In [370]:
df_test['result_'] = df_test['img_path'].apply(lambda x: predict(x))
df_test['result'] = df_test['result_'].apply(lambda x: x[1].item())
df_test['score'] = df_test['result_'].apply(lambda x: x[0])

In [371]:
df_test['score'].iloc[0]

tensor([0.9983, 0.0017], grad_fn=<SoftmaxBackward0>)

In [372]:
df_test.iloc[0]

Unnamed: 0                                                    0
image_name                                frame0vid1_1_11_1.jpg
label                                                       1.0
img_path      /Users/aliturliyeva/Documents/GitHub/patchnet_...
result_       [[tensor(0.9983, grad_fn=<UnbindBackward0>), t...
result                                                        0
score         [tensor(0.9983, grad_fn=<UnbindBackward0>), te...
Name: 0, dtype: object

In [373]:
df_test.to_csv('test_patchnet_result.csv')

In [374]:
df_test['result'].value_counts()

result
0    53
1    51
Name: count, dtype: int64

In [375]:
df_test['label'].value_counts()

label
1.0    52
0.0    52
Name: count, dtype: int64

In [376]:
y_test = df_test['label'] 
y_pred = df_test['result'] 

In [377]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")

classification_rep = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_rep)

confusion_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_mat)


Accuracy: 0.952
Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.96      0.95        52
         1.0       0.96      0.94      0.95        52

    accuracy                           0.95       104
   macro avg       0.95      0.95      0.95       104
weighted avg       0.95      0.95      0.95       104

Confusion Matrix:
[[50  2]
 [ 3 49]]
